# Virtual Machine Workload Characterization  



## VMware and virtualization

_"We believe that software has the power to unlock new possibilities for people and our planet. Our software forms a digital foundation that powers the apps, services, and experiences transforming the world."_

VMware is an IT company – leader in the server virtualization market. Virtualization is the process of running multiple instances of different computer system in a software layer abstracted from the actual hardware. Most commonly, it refers to running multiple operating systems on a single physical server simultaneously. To the applications running on top of the virtualized machine are not aware that they run on a virtual machine.

## Problem description

As two of the most-mature products, vSphere and vCenter Server provides great deal of customization and flexibility. However, given the complexity of the modern virtualization and the numerous different technologies involved in computing network and storage the customization options for VMware products grew infeasible large for common system administrators to grasp.

At the same time different workloads have different needs. There always is some tradeoff between different functionalities of the system (like throughput and latency or  consolidation and redundancy) and there is not one configuration to serve equally well all kind of workloads. Thus, understanding the purpose of the environment is crucial. 

And while it is easy to profile and optimize single virtual machine, vSphere stack hosts millions virtual environments. In order to approach their needs proactively we have to find a data driven way to classify them.  
</br> 

### The Challenge 

vSphere stack enables (with the explicit agreement of the owners of the environment) the collection of on demand low level performance telemetry. Based on this data we need to identify groups of virtual machines similar with respect to different properties, such as scale, utilization pattern etc. However, we will diverge from the standard clustering algorithms (although we will use one as supporting tool) and try to achieve this through embeddings.  

### But what is an embedding? 

Embedding is a representation of high dimensional vector on low dimensional space. Ideally this representation preserves as much information from the original vector by positioning similar inputs close together on the embedding space.

</br> 
### The Dataset 

The dataset consists of two main data sources related to the performance and the virtual hardware of the virtual machines (VMs).

The **performance telemetry** is organized in a python list, containing multiple python dictionaries. Each dictionary accounts for the data of single VM. The key of the dictionary is the respective ID of the VM, and the value is pandas data frame indexed by the timestamp and containing the actual measurements of each feature.  

</br>

Variable Name |Df index| type|unit | Description|
--- | --- |--- | --- |---
ts  |yes|timestamp|time|Time stamp (yyyy-mm-dd HH:MM:SS) of the observation|
cpu_run  |no|numeric|milliseconds|The time the virtual machine use the CPU|
cpu_ready|no|numeric|milliseconds|The time the virtual machine wants to run a program on the CPU but waited to be scheduled|
mem_active|no|numeric|kiloBytes|The amount of memory actively used by the vm|
mem_activewrite|no|numeric|kiloBytes|Amount of memory actively being written to by the virtual machine.|
net_packetsRx|no|numeric|count|Number of packets received during the interval.|
net_packetsTx|np|numeric|count|Number of packets transmitted during the interval.|

</br> 
The **virtual hardware dataset** is a normal rectangular data frame indexed by the id of the VM. It represents “static” features that account basically for the scale of the system.  
</br>  
  

Variable Name |Df index| type|unit | Description|
--- | --- |--- | --- |---
id  |yes|integer| |Unique identifier of the virtual machine |
memory_mb|no|integer|megabytes|Configured virtual RAM|
num_vcpus|no|integer|count|Number of virtual processor cores|
number_of_nics|no|integer|count|Number of network interface cards|
num_virtual_disks|no|integer|count|Number of the configured hdd|
os_fam|no|categorical|indetity|The operating system of the VM|

</br></br>


## Environment setup

### Before we begin, we should do some groundwork:

*  Doing the initial imports
*  Mounting Google Drive as our file system
*  Setting some path variables and creating working directory 
*  Download the data from external repo (another location in Google Drive)  
*  Define some functions that we will reuse many times
   

In [0]:
## SETUP THE ENVIRONMENT 

## Import general system modules
from google.colab import drive
from google_drive_downloader import GoogleDriveDownloader
from itertools import product
from itertools import chain
import random 
import pickle
import gzip
import sys
import os

## Core modules
import pandas as pd
import numpy as np

## Import plotting libraries
from matplotlib import pyplot as plt
import matplotlib 
import seaborn as sns

## ML modules
from sklearn.manifold import TSNE
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA 
from sklearn.cluster import DBSCAN
import umap

## Plot functions display options 
%matplotlib inline
sns.set(rc={'figure.figsize':(11.7,8.27)})

In [0]:
## Mount your google drive as file system
## You should allow this notebook to access your drive
## ultimately receiving temporary token
drive.mount('/content/drive')

In [0]:
## Setup our root dir
gdrive_root = "/content/drive/My Drive/AMLD VMWARE/" 
performance_file_name = 'performance_telemetry.gzip' 
viritual_hardware_file_name = 'virtual_hardware.csv'
utilities_file = 'workshop_utilities.py'

## Construct the file path of the file we are going to download and use: 
performance_data_full_path = gdrive_root + performance_file_name
virtual_hardware_data_full_path = gdrive_root + viritual_hardware_file_name
utilities_file_full_path = gdrive_root + utilities_file


## Check if the default working directory exist and if no, create it: 
if not(os.path.exists(gdrive_root)):
    os.mkdir(gdrive_root)
    print(gdrive_root,' directory has been created!')

## Add the new path to the runtime
## From here we will import some custom functions
sys.path.append(gdrive_root)

## Set the root dir as default
os.chdir(gdrive_root)
print('File paths have been set!')

In [0]:
## This chunk downloads data from our shared storage and makes a "local copy to your personal google drive"
## After the first execution of this chunk the data is cached and in the next run it won’t download the data. 
## In order to execute it again, you have to delete the files from the folder (or the entire folder) 


## Files location
performance_tm_fileid = '1tQulULMw09XQ5rWKF0fqEzXHRwX2CwBS'
virtual_hw_fileid = "136ZxWcIflPD_yol5Nhkj9LVeYB9-1W6L"
workshop_util_file = '1amC8ZhG7ZQt1fwJoidLC8GF58yXTQrJz'

## Download Data
GoogleDriveDownloader.download_file_from_google_drive(file_id = performance_tm_fileid , dest_path = performance_data_full_path     )
GoogleDriveDownloader.download_file_from_google_drive(file_id = virtual_hw_fileid     , dest_path = virtual_hardware_data_full_path)
GoogleDriveDownloader.download_file_from_google_drive(file_id = workshop_util_file    , dest_path = utilities_file_full_path       )

### Print the content of our working folder
#it should contain ['performance_telemetry.gzip', 'virtual_hardware.csv', 'workshop_utilities.py']
print('Dir content:', os.listdir(gdrive_root))

In [0]:
## Import some predefined utility functions
from workshop_utilities import truncate_by_quantile, plot_single_vm, sample_observation, model_params_product, model_train, plot_results, plot_embedding
print('Custom utility functions loaded!')

In [0]:
## Load data in memory for this session: 

## Load performance data
with gzip.open(performance_file_name, 'rb') as fconn:
    perf_data = pickle.load(fconn)
    print('Performance Telemetry successfully loaded !')

## Load virtual hardware
virt_hw = pd.read_csv(viritual_hardware_file_name, sep = ';', index_col = 'id')
print('Virtual Hardware successfully loaded !')

In [0]:
## Define function that applies numeric transformation 
def return_transformer(transformer = None, numeric_constant = 1, lambd = 0):
    if transformer == None or transformer == 'None':
        return(lambda x: x)
    elif transformer == 'ln' or (transformer=='boxcox' and lambd == 0):
        return(lambda x: np.log(x+numeric_constant))
    elif transformer == 'log10':
        return(lambda x: np.log10(x+numeric_constant))
    elif transformer=='boxcox':
        return(lambda x: (np.power(x+numeric_constant, lambd) - 1)/lambd ) 

    ### ADD YOUR OWN TRANSFORMATION HERE 

In [0]:
### To run all code chunks go to Runtime>Run before (Ctrl + F8)

### Env setup end

## First look at our data

In [0]:
## Very basic data exploration 
## Define some global variables that we will reuse 
##   N: Number of virtual machines 
##   D: Number of performance features 
##   T: Temporal dim: number of time samples
##   F: List of features 
## VMs: List with all ids of the virtual machines

N = len(perf_data)
VMs = list(perf_data.keys())
T , D = perf_data[VMs[0]].shape
F = list(perf_data[VMs[0]].columns)


print('Dictionary with', N ,' virtual machines!')
print('Each VM has ',T, ' observations in time!')
print('There are',D, 'performance features:',  F)
print('\n First rows of the first virtual machine \n')
pd.set_option('display.max_columns', 10)
print(perf_data[1].head())

In [0]:
##   D_hw: Number of performance features 
##   F_hw: List of features  

_ , D_hw = virt_hw.shape
F_hw = list(virt_hw.columns)

print('Dictionary with', N ,' virtual machines!')
print('There are',D_hw, 'virtual hardware features:',  F_hw, '\n')

print('Missing values:')
print(np.sum(virt_hw.isna()))

print('\nData types:')
print(virt_hw.dtypes)

print('\nData Frame Summary:')
virt_hw.describe()

print(virt_hw.head())

In [0]:
fig, axes = plt.subplots(nrows = 2, ncols=3, sharex = False, sharey = False, figsize = (40, 10))


for i, metric in enumerate(F_hw):
    
    ax = axes.reshape(-1)[i]
    data = virt_hw.groupby([metric])[metric].count()
    x_ax = [str(i) for i in data.index.values]
    ax.bar(x_ax, height  = data.values)
    ax.set_title(metric)

### Inspect single VM series

In [0]:
## 
DATA = perf_data
VMID = None ## None for random VM 
TRANSFORMATION = return_transformer(None)


###
plot_single_vm(__perf_data = DATA
             , __vmid = VMID
             , transformation = TRANSFORMATION
             )

### Note on data transformation 

Transforming variable means replacing the actual value x with some function of that variable f(x). As a result, we change the distribution and the relationship of this variable in some beneficial way. 

Here we will consider two basic techniques:
*  Box-Cox(power) transformation - This is a useful data transformation technique used to stabilize variance, make the data more normal distribution-like, improve the validity of measures of association.

*  Logarithmic transformation - logarithmic function has many nice properties that are commonly exploited in the analysis of time series. Change of natural log roughly equals percentage change. It also converts multiplicative relationship into additive (log(XY) = log(X) + log(Y)).  Natural log transformation is a private case of Box-Cox transformation
  

*  Data truncation - many of the statistics, that we are going to compute are quite sensitive to extreme values and outliers. Thus, it might be worth sacrificing tiny portion of the data in order to limit the effect of extreme values on our estimates. Note that such transformation might obscure the actual distribution and should be applied with care.

In [0]:
################
## PARAMETERS ##
################

VMID = None                # None or integer between 1 and 707 
METRIC = 'mem_activewrite' # If None then random metric is chosen
TRUNCATE_VARIABLE = True # 
TRUNC_QRANGE = [0,0.99]     # no truncation =[0,1]
TRANSFORMATION = return_transformer(None) ## returns lambda expression. Arguments:None, 'ln', log10,  'boxcox' + lambda


##############
if METRIC == None:
    METRIC = F[np.random.randint(0, D)]

VMID, data = sample_observation(perf_data, vmid = VMID)


## Inspect the distribution of the variable: 
fig = plt.figure(figsize=(20,10))
gs0 = fig.add_gridspec(2,1)
gs1 = gs0[1].subgridspec(1,2)


ax0 = fig.add_subplot(gs0[0])
ax1 = fig.add_subplot(gs1[0])
ax2 = fig.add_subplot(gs1[1])
ax1.set_title('Untransformed distribution')
ax2.set_title('Transformed distribution')

ax0.set_title('Series in time {feat} for VMID {vmid}:'.format(feat = METRIC, vmid = VMID))
ax0.plot(data.index,  data[METRIC], linewidth = 0.9)
ax0_0 = ax0.twinx()
ax0_0.plot(data.index, TRANSFORMATION(data[METRIC]), label = 'transformed', c = 'orange', linewidth = 0.5)
## First plot the original series and then truncate the data 

data['truncated'] = data[METRIC]
if TRUNCATE_VARIABLE:
    #data = data.apply(lambda x: truncate_by_quantile(x, trunc_range = TRUNC_QRANGE))
    data['truncated'] = truncate_by_quantile(data[METRIC], trunc_range = TRUNC_QRANGE)

sns.distplot(data[METRIC].dropna(), kde = False, ax = ax1, bins = 40)
sns.distplot(TRANSFORMATION(data['truncated'].dropna()), kde = False, ax = ax2, bins = 40 )

ax1.set_xlabel(None)
ax2.set_xlabel(None)
plt.legend()
plt.show()

### Which transformation? 
Among thigs to consider when you transform your variables are what works with the data, does it makes sense and is there a natural interpretation after transforming the data?  
</br>

**References:**  
[Transformations: an introduction with STATA](http://fmwww.bc.edu/repec/bocode/t/transint.html)  
[The logarithm transformation](https://people.duke.edu/~rnau/411log.htm)  
[MAKING DATA NORMAL USING BOX-COX POWER TRANSFORMATION](https://www.isixsigma.com/tools-templates/normality/making-data-normal-using-box-cox-power-transformation/)  
[Yeo-Johnson Power Transformation](https://www.stat.umn.edu/arc/yjpower.pdf)


</br> 
**Some insights regarding the performance telemetry:**

*  Our data is strictly non-negative (convenient for power transformations)  
*  Most of the performance metrics do not exhibit clear trend
*  Missing data due do data collection routine 
*  There are instances with multimodal distribution 



## Feature engineering: Basic descriptives


The first stage of our FE is to obtain very basic descriptive statistics. Descriptive statistics are coefficients that summarize some aspect of the empirical distribution of a variable. The most common aspects that are measured by the descriptive statistics are the central tendency (mean, median, mode) and variability (standard deviation, variance, kurtosis and skewness).
<br></br>
**_PROS:_**   
*  The descriptive are not as sensitive to relatively small amounts of missing data   
*  Easy and fast to compute and often comes already implemented in the most software packages   
*  Easy to interpret   


 **_CONS:_**   
*  They are not very expressive and can hide a lot of the information  
*  Some of the basic descriptives are very sensitive to extreme values and need some preprocessing
<br></br>

In order to mitigate the effect of extreme values we will do some statistical tricks, including truncating all values past a threshold and transforming our data on another scale. This way we will improve the statistical properties of our data and make our estimates more robust.

### How will we create our initial features?  

1.  Apply numeric transformation that will change the scale and the distribution of the initial data
2. Apply multiple function that takes one or more pd.Series as argument and yields single numerical representation of this / these series. Store them in some data structure  
3. Flatten this data structure into vector  for each virtual machine  

Ultimately, we are going to end with tidy rectangular dataset where each row represents single virtual machine and each column represents feature that we have extracted from our original data.  

[Pandas Series Methods](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.html)

In [0]:
## How can we apply custom functions ? 
## Define function that operates on pd Series

def missing_count(series):
    return(len(series) - series.count())

def Q25(series):
    return(series.quantile(0.25))

def Q50(series):
    return(series.quantile(0.50))

def Q75(series):
    return(series.quantile(0.75))

## ADD YOUR CUSTOM FUNCTIONS HERE

In [0]:
## Single case example: 

## Apply multiple build in aggregation functions with relatively fast implementation 
## the available functions are sum, mean, min, max, var, sem, skew, kurt, count 
## sem - standard error of the mean
## kurt - kurtosis (4th moment)
## skew - skewness (3rd moment)
## var  - variance (2nd moment)
## These functions can be passed as an array to each column of the data frame: 

_, data  = sample_observation(perf_data)
np.round(data.agg(['mean', 'min', 'max', 'var', 'skew', Q50,missing_count]), 3)

In [0]:
## Correlation matrix
data.corr()

#### Creating the features
</br> 
By the end of our feature engineering section we will end up with single data frame that combines the information from both descriptive statistics and correlation data.  
*  Our initial datasets will be called  **descriptive_statistics** and **correlation_data**. There is a pandas view that consolidates them into single data frame called **non_temporal_features**  
*  The features in this data set can be referenced by the column names
  * The names of the features, yielded from the descriptive statistics analysis are stored in **descriptive_statistics_columns** variable
  * The names of the features, yielded from the correlation matrix are stored in **correlation_features_columns** variable  


In [0]:
## FEATURE GENERATION CHUNK ##
## FOR EACH VM
##     FOR EACH METRIC [cpu_run, cpu_ready ... net_packetsRx]
##          1. TRUNCATE BY QUANTILE 
##          2. APPLY TRANSFORMATION
##          3. COMPUTE EACH METRIC IN GENERAL_STATISTICS 
##          5. CREATE FEATURE VECTOR WITH COLUMNS {METRIC}_{STATISTIC} [cpu_run_mean, cpu_run_std, .... ]
##          4. COMPUTE CORRELATION 
##          5. FOR EACH UNIQUE PAIR OF METRICS CREATE FEATURE VECTOR [corr_cpu_run_cpu_ready, corr_cpu_run_mem_active, ... ]
## 6. COMBINE THE INFORMATION FOR EACH VIRTUAL MACHINE IN TWO DATA FRAMES   



GENERAL_STATISTICS = ['mean','std','skew',Q50, Q25, Q75]
TRANSFORMATION =  return_transformer('None')
TRUNCATE_VARIABLE = True
TRUNC_QRANGE = [0.001, 0.999]




#######################
## INIT PLACEHOLDERS ##
#######################

descriptive_statistics = []
correlation_data = []
i = 1
for k, data in perf_data.items():

    ###################
    ## TRUNCATE DATA ##
    ###################

    if TRUNCATE_VARIABLE:
        data = data.apply(lambda x: truncate_by_quantile(x, trunc_range = TRUNC_QRANGE))
    
    ## Apply transformation
    data = TRANSFORMATION(data)

    ############################
    ## Descriptive statistics ##
    ############################

    ## Apply aggregation functions
    general_descriptives = data.agg(GENERAL_STATISTICS)
    
    ## apply modifications to general descriptives: 
    ## 1. converts the data frame to series with index [(summary_statistic,performance_metric)]
    general_descriptives = general_descriptives.stack()

    ## Convert 2d index to 1d by string concat: 
    general_descriptives.index = general_descriptives.index.map('{0[1]}_{0[0]}'.format)

    ## Convert the series to data frame and transpose it 
    general_descriptives = general_descriptives.to_frame().T

    ## Add the VM indentifier: 
    general_descriptives['vmid'] = k

    ########################
    ## Correlation matrix ##
    ########################

    # Calculate correlation matrix:  
    correlation_matrix = data.corr()
    corr_df_tmp = pd.DataFrame(index=[k])


    ## Iterate over the elements of the (cross) correlation matrix
    ## Take the elements below the diagonals 
    ## This way we flatten the correlation matrix into feature vector: 
    
    for mrow in range(D):
        for mcol in range(D):
            if mrow >= mcol:
                continue

            # Construct new col name:
            new_col = 'corr_' + F[mrow] + '_' + F[mcol]
            corr_df_tmp[new_col] = correlation_matrix.iloc[mrow,mcol]
    
    if i%100 == 0:
        print('Finished iteration', i, 'out of', N)
    i += 1
    
    ## Finally set vmid as index (on the fly)
    ## Join correlation data with descriptive statistics:
    ## as add the record to the data placeholder:

    descriptive_statistics.append(general_descriptives.set_index('vmid'))
    correlation_data.append(corr_df_tmp)

print('Finished iteration', N, ' out of ', N)

## The loop left us with list of single row data frames both for descriptive statistics and correlation matrix
## Next step is to combine the whole data into one big data frame  
descriptive_statistics = pd.concat(descriptive_statistics)
correlation_data = pd.concat(correlation_data)

## We will further process the different families of features 
## Thats why we keep their names in separate variables 
descriptive_statistics_columns = descriptive_statistics.columns
correlation_features_columns = correlation_data.columns

## Finally merge the final dataset into one 
non_temporal_features = descriptive_statistics.merge(correlation_data, left_index = True, right_index= True)
non_temporal_features_index = non_temporal_features.index

## Small trick to ensure alignment between our features and the virtual hardware data
virt_hw = virt_hw.loc[non_temporal_features.index]

#### Inspect the features
Visualize the distributions of the features that we’ve created. Please keep in mind the transformation that we’ve applied over the original series.

In [0]:
## Inspect the summary statistics of the correlation features
pd.set_option('display.max_columns', 50)
np.round(correlation_data.describe(), 2)


,corr_cpu_run_cpu_ready,corr_cpu_run_mem_active,corr_cpu_run_mem_activewrite,corr_cpu_run_net_packetsRx,corr_cpu_run_net_packetsTx,corr_cpu_ready_mem_active,corr_cpu_ready_mem_activewrite,corr_cpu_ready_net_packetsRx,corr_cpu_ready_net_packetsTx,corr_mem_active_mem_activewrite,corr_mem_active_net_packetsRx,corr_mem_active_net_packetsTx,corr_mem_activewrite_net_packetsRx,corr_mem_activewrite_net_packetsTx,corr_net_packetsRx_net_packetsTx
count,707.00,707.00,707.00,707.00,707.00,707.00,707.00,707.00,707.00,707.00,707.00,707.00,707.00,707.00,707.00
mean,0.39,0.21,0.21,0.33,0.35,0.06,0.07,0.22,0.16,0.62,0.10,0.14,0.11,0.14,0.67
std,0.42,0.27,0.26,0.36,0.36,0.19,0.19,0.34,0.31,0.18,0.20,0.22,0.20,0.21,0.36
min,-0.98,-0.62,-0.58,-0.80,-0.65,-0.87,-0.87,-0.77,-0.79,0.15,-0.36,-0.35,-0.39,-0.21,-0.31
25%,0.12,0.03,0.03,0.07,0.09,-0.02,-0.01,-0.00,-0.02,0.50,0.00,0.01,0.00,0.01,0.34
50%,0.41,0.12,0.13,0.22,0.28,0.02,0.03,0.09,0.05,0.58,0.04,0.06,0.04,0.07,0.83
75%,0.75,0.38,0.34,0.56,0.59,0.08,0.11,0.38,0.25,0.73,0.13,0.20,0.15,0.19,0.97
max,1.00,0.96,0.96,1.00,1.00,0.95,0.95,0.99,0.97,1.00,0.89,0.91,0.93,0.92,1.00


In [0]:
TRANSFORMATION = return_transformer(None)

## Inspect the distribution of cross correlation
g = sns.FacetGrid(pd.melt(TRANSFORMATION(correlation_data)), col = 'variable', col_wrap = 5, aspect = 1.5)
g.map(plt.hist, 'value', bins = 40)
plt.subplots_adjust(top=0.9)
g.fig.suptitle('CORRELATION FEATURES')


We can see that the distribution  of the correlation is skewed towards the positive correlations . There are several cross-correlation features with bi-modal distributions.


In [0]:
## Summary of the descriptive statistics features
pd.set_option('display.max_columns', 50)
print('Min feature value is:', np.min(np.round(descriptive_statistics.describe(), 1).values))
np.round(descriptive_statistics.describe(), 1)


In [0]:
## Inspect the distribution of the descriptive statistics 
TRANSFORMATION = return_transformer('ln', numeric_constant=5)

#######
## Transform the data from wide to long format 
data = TRANSFORMATION(descriptive_statistics)
data = pd.melt(data)
#data['variable']  = TRANSFORMATION(data['variable'])

## Plot 
g = sns.FacetGrid(data, col = 'variable', col_wrap = 6, sharex = False, sharey=False, aspect = 1.5)
g.map(plt.hist, 'value', bins = 40)
plt.subplots_adjust(top=0.9)
g.fig.suptitle('DESCRIPTIVE FEATURES')

#### Apply transformation

Each of the following chunks creates a data frame where it applies transformations. We can either choose to leave the data untransformed, to standardize (subtract mean and divide by std) or to normalize data within some numeric range (typically 0, 1 or -1,1).
Typically, is not good idea to apply different scaling to the different features.  A rule of the thumb (at least for this workshop) is to have up to one transformation and one scaling.


##### Descriptive statistics transformations

In [0]:
############################
##:: DATA PREPROCESSING ::##
##::    DESCRIPTIVES    ::##
############################

## Apply transformation
TRANSFORMATION = return_transformer(None)

APPLY_STANDARD_SCALING = False

APPLY_MIN_MAX_SCALING = True
MIN_MAX_SCALER_RANGE = (-1, 1)


##############
## We would like to retain the original features
## Thus we create hard copy of the data and operate on it:
descriptive_statistics_transformed = descriptive_statistics.copy()

## Rescale descriptive statistics data to within  MIN MAX SCALER RANGE 
## The correlation features are naturally scaled within this range 
min_max_scaler_instance = MinMaxScaler(feature_range = MIN_MAX_SCALER_RANGE)
standart_scaler_instance = StandardScaler()

## Apply transformation
descriptive_statistics_transformed[descriptive_statistics_columns] = TRANSFORMATION(descriptive_statistics_transformed[descriptive_statistics_columns])


## APPLY MIN-MAX SCALING TO DESCRIPTIVE STATISTICS DATA
if APPLY_MIN_MAX_SCALING:
    descriptive_statistics_transformed[descriptive_statistics_columns] = min_max_scaler_instance.fit_transform(descriptive_statistics_transformed[descriptive_statistics_columns])

## APPLY MIN-MAX SCALING TO DESCRIPTIVE STATISTICS DATA
if APPLY_STANDARD_SCALING:
    #modeling_data[STANDARD_SCALING_COLUMNS] = standart_scaler_instance.fit_transform(modeling_data[STANDARD_SCALING_COLUMNS])
    descriptive_statistics_transformed[descriptive_statistics_columns] = standart_scaler_instance.fit_transform(descriptive_statistics_transformed[descriptive_statistics_columns])

##### Correlation features transformations

In [0]:
############################
##:: DATA PREPROCESSING ::##
##::     CORRELATION    ::##
############################

## Apply transformation
TRANSFORMATION = return_transformer(None)

APPLY_STANDARD_SCALING = False

APPLY_MIN_MAX_SCALING = False
MIN_MAX_SCALER_RANGE = (-1, 1)

###############
## We would like to retain the original features
## Thus we create hard copy of the data and operate on it:
correlation_data_transformed = correlation_data.copy()

## Rescale descriptive statistics data to within  MIN MAX SCALER RANGE 
## The correlation features are naturally scaled within this range 
min_max_scaler_instance = MinMaxScaler(feature_range = MIN_MAX_SCALER_RANGE)
standart_scaler_instance = StandardScaler()

## Apply transformation
correlation_data_transformed[correlation_features_columns] = TRANSFORMATION(correlation_data_transformed[correlation_features_columns])


## APPLY MIN-MAX SCALING TO DESCRIPTIVE STATISTICS DATA
if APPLY_MIN_MAX_SCALING:
    correlation_data_transformed[correlation_features_columns] = min_max_scaler_instance.fit_transform(correlation_data_transformed[correlation_features_columns])

## APPLY MIN-MAX SCALING TO DESCRIPTIVE STATISTICS DATA
if APPLY_STANDARD_SCALING:
    #modeling_data[STANDARD_SCALING_COLUMNS] = standart_scaler_instance.fit_transform(modeling_data[STANDARD_SCALING_COLUMNS])
    correlation_data_transformed[correlation_features_columns] = standart_scaler_instance.fit_transform(correlation_data_transformed[correlation_features_columns])

##### Virtual hardware transformations

In [0]:
###############################
###:: DATA PREPROCESSING ::####
###::VIRTUAL HARDWARE DATA::###
###############################

## Apply transformation
TRANSFORMATION = return_transformer(None)

APPLY_STANDARD_SCALING = False

APPLY_MIN_MAX_SCALING = True
MIN_MAX_SCALER_RANGE = (-1, 1)


## Make copy of the original data
virt_hw_transformed = virt_hw.copy()
virt_hw_transformed_columns = virt_hw_transformed.columns

min_max_scaler_instance = MinMaxScaler(feature_range = MIN_MAX_SCALER_RANGE)

## One hot encode categorical 

virt_hw_numerics = []
for col in virt_hw_transformed_columns:
    col_type = virt_hw_transformed[col].dtypes.name
    
    if col_type == 'object' or col_type == 'category':
        dummy_vars = pd.get_dummies(virt_hw_transformed[col], prefix = col)
        virt_hw_transformed = pd.concat([virt_hw_transformed,dummy_vars],axis=1)
        virt_hw_transformed.drop([col],axis=1, inplace=True)
    else:
        virt_hw_transformed[col] = TRANSFORMATION(virt_hw_transformed[col])
        virt_hw_numerics.append(col)

if APPLY_MIN_MAX_SCALING:
    min_max_scaler_instance = MinMaxScaler(feature_range = MIN_MAX_SCALER_RANGE)
    virt_hw_transformed[virt_hw_numerics] = min_max_scaler_instance.fit_transform(virt_hw_transformed[virt_hw_numerics])

if APPLY_STANDARD_SCALING:
    standart_scaler_instance = StandardScaler()
    virt_hw_transformed[virt_hw_numerics] = standart_scaler_instance.fit_transform(virt_hw_transformed[virt_hw_numerics])


##### Assemble the dataset for modeling

In [0]:
## CREATE FINAL DATASET ## 
## CHOOSE WHICH FEATURES TO INCLUDE FOR THE FINAL DATASET
## BY DEFAULT, WE WILL MODEL ALL FEATURES THAT WE’VE CREATED 
## HOWEVER, WE CAN MAKE MODELS OUTPUT EASIER TO INTERPRET IF WE USE INFORMATION ONLY FOR ONE RESOURCE TYPE (CPU or NETWORK) 


## To include columns that contains cpu_run
## descriptive_statistics_transformed.filter(regex = 'cpu_run')

## To exclude columns containing cpu_run string 
## descriptive_statistics_transformed.loc[:,~descriptive_statistics_transformed.columns.str.contains('cpu_run')]


first_stage_data_list = [
                              descriptive_statistics_transformed
                             ,correlation_data_transformed
                             ,virt_hw_transformed
                            ]




## Init empty data frame: 
first_stage_data = pd.DataFrame(index = non_temporal_features_index)

## 
for i in first_stage_data_list:
    #data = i.copy() ## in the cases where there will be more transformations 
    data = i  
    first_stage_data = pd.merge(first_stage_data, data, left_index=True, right_index=True)

print('Final dataset is with shape:', first_stage_data.shape)

In [0]:
### To run all code chunks go to Runtime>Run before (Ctrl + F8)

## Principal component analysis

Although PCA can be though as form of embedding by itself, despite all benefits (speed, efficiency, interpretability), it suffers from a major drawback. It is tricky to capture (by default) non-linear relationships.   
</br> 
**References:**  
https://towardsdatascience.com/a-one-stop-shop-for-principal-component-analysis-5582fb7e0a9c  
http://setosa.io/ev/principal-component-analysis/  
[Making sense of principal component analysis, eigenvectors & eigenvalues](https://stats.stackexchange.com/questions/2691/making-sense-of-principal-component-analysis-eigenvectors-eigenvalues)




In [0]:
from sklearn.decomposition import PCA 

In [0]:
## Plot correlation matrix
cmap = sns.diverging_palette(220, 20, sep=20, as_cmap=True)
sns.clustermap(first_stage_data.corr(), figsize= (20,20), cmap = cmap)
plt.show()

In [0]:
## One more thing to consider is the correlation among the variables: 
pca_instance = PCA().fit(first_stage_data)

## Taking care to preserve the index for later join: 
first_stage_data_pca = pd.DataFrame(pca_instance.transform(first_stage_data), index = first_stage_data.index)

fig, ax = plt.subplots(figsize=(20,8),nrows = 1 , ncols = 2)

sns.scatterplot(x = first_stage_data_pca.iloc[:,0], y = first_stage_data_pca.iloc[:,1], ax = ax[0])
ax[0].set_title('Principal Components')
ax[0].set_xlabel("PC 1")
ax[0].set_ylabel("PC 2")

ax[1].bar(x = [i+1 for i in range(pca_instance.n_components_)], height  = pca_instance.explained_variance_/ np.sum(pca_instance.explained_variance_))

ax2 = ax[1].twinx()
ax2.plot([i+1 for i in range(pca_instance.n_components_)],  np.cumsum(pca_instance.explained_variance_)/ np.sum(pca_instance.explained_variance_), c = 'orange')
ax2.grid([])
ax2.set_ylim(bottom =0)

ax[1].set_title('Explained variance plot:')
ax[1].set_xlabel('Number of principal components')
ax[1].set_ylabel('Explained variance')
ax2.set_ylabel('Cumulative variance')
## 

The easiest way to reduce the redundancy of our data is to apply PCA.  

PCA is beneficial preprocessing step for t-SNE, recommended both by the author and by the ML practitioners. 
  
Number of components to include is a tunable parameter by itself and initially we can adopt the “elbow” approach. Generally we should identify where the marginal explained variance starts to diminish.  



## t-Distributed Stochastic Neighbor Embedding (t-SNE)

Intended as a technique for dimensionality reduction well suited for the visualization of high-dimensional datasets

**Materials**  
[Official Website With papers and implementations under different languages](https://lvdmaaten.github.io/tsne/)   
[How to Use t-SNE Effectively](https://towardsdatascience.com/how-exactly-umap-works-13e3040e1668) an amazing interactive blogpost  
[How to tune hyperparameters of tSNE](https://towardsdatascience.com/how-to-tune-hyperparameters-of-tsne-7c0596a18868)  
[sklearn implementation](https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html)


In [0]:
from sklearn.manifold import TSNE

In [0]:
## Simple t-SNE usage: 
## Tsne follows the sklearn interface :

# 1. We create instance of the object
# Here we set the parameters that are internal to the algorithm

TSNE_Instance = TSNE(n_components =  2          ## Controls the number of dimebsuibs if the embedded space
                    ,perplexity = 10            ## Perplexity controls how to balance attention between local and global aspects of your data 
                    ,early_exaggeration = 12    ## Controls how tight natural clusters are situated
                    ,learning_rate = 100        ## Controls the step during the gradient descent stage
                    ,n_iter = int(1e3)          ## Max number of iterations
                    ,random_state =  500        ## Random seed 
                    ,verbose = 2                ## controls logging of the algorithm
                     )

# 2.Fit the object to the data 
TSNE_results = TSNE_Instance.fit_transform(first_stage_data)

# The result of fit_transform is ndarray with shape (NSamples, n_components)

plt.figure(figsize=(7,7))
plt.scatter(TSNE_results[:, 0], TSNE_results[:, 1])
plt.xlabel('X')
plt.ylabel('Y')


### t-SNE tunable parameters
**n_components** - similar to N components In PCA. It sets the cardinality of your output feature set. sklearn t-sne implementation supports up to 3d plane. 

_Practical note:_ algorithm becomes exponentially slow with the increase of this parameter  
  
**perplexity** - Perplexity controls how to balance attention between local and global aspects of your data. Low values yield many small scattered clusters while large values tend to clump up the data. This parameter is very data dependent and it should be explored every time 

**PCA_components** - Although not internal for the algorithm in practice t-SNE benefits and in a way complements PCA preprocessing. With the former the linear structure of the data is captured while with the former technique can capture non-linear latent structure.

 

### Technical note on the utility functions
Since we’ve intended this workshop to be more of a decision making exercise rather that coding one, we are about to introduce several utility functions, defined in order to visualize and automate some of the routines that about to go through.  
</br>


#### model_train()

**model_train** :  this function will be used to train the embedding with multiple predefined values of the tunable parameters:
*  data - pandas data frame or numpy matrix. The data is restricted to numeric arrays without missing observations
*  model - string with one of the following values: _"TSNE", "DBSCAN", "UMAP"_
* param_dict - python dictionary with tunable parameters. The name of the parameter must be the key of the dictionary, the value must be **python list** with values.  
</br>

       param_dict = {
                     'perplexity':[10,20]
                    , n_iter:[300,500] 
                    }

*  apply_pca - boolean parameter that indicates if the original data will be PCA transformed. 
*  n_pc - Number of principal components applied as **list of integers** (n_pc = [10, 20]). This parameter is considered as tunable and it is added under the tunable parameters in the output of the function. It is ignored if apply_pca = False
* other_params - non-tunable parameters. Like __param_dict__ the keys should be the name of the parameter. Unlike __param_dict__ the value should not be list
</br>

       param_dict = {
                     'learning_rate':np.float32(1e-3)
                    }

The result of the function is python list. Each element contains triple object where:
*  First element of the triple is the dictionary with tunable parameter values
*  The second element is the coordinates of the embedding
*  The last element is the non-tunable parameters 

This result is fed to several plotting  functions in order to produce neat visualizations that will help us interpret the results of our embedding

####plot_results()  
This is one of the first plotting function that is designed for exploration of the tuning results. The way it works it that it takes object, generated  from model_train() function and creates grid of plots arranged by the values of the parameters that have been used for the embeddings.  

Important to note is that this function **plots only the first two dimensions** of the embedding.  If the number of dimensions is higher than 2 plot_embedding() can be used to visualize all dimensions. 

**Arguments:**  
*  results - results, generated from model_train()  
*  leading_param = None - the name of the parameter that should be used to arrange the tuning results row wise.
*  color_array - list of numpy arrays that will be used to apply color to the scatterplot matrix. If given, the length of the list should be either 1 (all scatters are colored with the same variable) or equal to the length of the results object (then each object is colored with its respective color array).  The second option will be used during exploring the results from DBSCAN clustering. 
* force_categorical = False - if the color array should be represented on a discrete color scale.  This option is meaningful for integer color arrays with limited number of distinct values 
* point_size = 3 -  controls the size of the points in the scatterplot

#### plot_embedding()

This is a function that is designed to plot single embedding. It takes **single** object, generated from model_train() and plots all dimensions are scatterplot matrix.  

**Arguments**  
* result - single element subset from model_train() results.
* fig_scale - plot scaling parameter
* color_var_list- list of numpy arrays that is used for coloring 
*  force_categorical = False - if the color array should be represented on a discrete color scale.  This option is meaningful for integer color arrays with limited number of distinct values 
* plot_centers = FALSE - used when we plot DBSCAN clusters. It overlays the cluster centers
* dont_plot_minus_one - used when we plot DBSCAN clusters. Don't plot outlier centers 
* point_size = 4 -  controls the size of the points in the scatterplot

### Tune T-SNE

In [0]:
## EXPERIMENT WITH DIFFERENT TSNE SETTINGS ## 


## TSNE PARAMETERS
## https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html

MODEL = 'TSNE'

TUNABLE_PARAMETERS = {'perplexity':[25]}
DEFAULT_PARAMETERS = {'learning_rate':np.int(1e2)}
DATA = first_stage_data

APPLY_PCA = True
N_PC = [30,50, 55]


## Apply tuning:
TSNE_results = model_train(   data  = DATA
                            , model = MODEL
                            , param_dict = TUNABLE_PARAMETERS
                            , apply_pca = APPLY_PCA
                            , n_pc = N_PC
                            , other_params = DEFAULT_PARAMETERS
                            )

In [0]:
## Inspect the results of the parameters

################
## PARAMETERS ##
################

RESULTS = TSNE_results
LEADING_PARAM = 'n_pc'
COLOR_ARRAY = None
FORCE_CATEGORICAL = False
POINT_SIZE = 5

## APPLY FUNCTION 
plot_results(results = RESULTS
            ,leading_param = LEADING_PARAM
            ,color_array = COLOR_ARRAY
            ,force_categorical = FORCE_CATEGORICAL
            ,point_size = POINT_SIZE
             )
## If single combination is being expected
##plot_embedding(RESULTS[0], fig_scale= 1.2,point_size = 5)

### Capture "clusters" through DBSCAN :

We will use Density-based spatial clustering of applications with noise (DBSCAN) algorithm to address the different "clusters" in our embedding. This is not a true clustering because t-SNE eliminated the actual structure of the data. However, this pseudo clustering step is useful and can guide us through the profiling of our final clusters

The tunable parameters that we are going to consider are: 

**eps:** specifies how close points should be to each other to be considered a part of a cluster. It means that if the distance between two points is lower or equal to this value (**eps**), these points are considered neighbors  

**min_samples:** the minimum number of points to form a dense region. For example, if we set the **min_samples** parameter as 5, then we need at least 5 points to form a cluster.  
</br>
**Several references:**  
[DBSCAN Original paper](http://www2.cs.uh.edu/~ceick/7363/Papers/dbscan.pdf)  
[Medium post](https://towardsdatascience.com/machine-learning-clustering-dbscan-determine-the-optimal-value-for-epsilon-eps-python-example-3100091cfbc)  
[sklearn implementation](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html)  



In [0]:
from sklearn.cluster import DBSCAN

In [0]:
## Choose one embedding to operate with 
## Zoom plot in : 
RESULTS = TSNE_results
MODEL_INDEX = 1


###
plot_embedding(RESULTS[MODEL_INDEX], fig_scale = 2, point_size = 9)

#### Tune DBSCAN

In [0]:
## Capture the clusters with dbscan 

TUNABLE_PARAMETERS = {'eps': [4,5,6]
                    , 'min_samples':[15,30,45]}
DEFAULT_PARAMETERS = {}
MODEL = 'DBSCAN'

## Apply tuning:
dbscan_clusters = model_train(  data = RESULTS[MODEL_INDEX][1]
                                , model = MODEL
                                , param_dict = TUNABLE_PARAMETERS
                                , other_params = DEFAULT_PARAMETERS
                                )

In [0]:
## Plot the results from DBSCAN clustering 
result_for_plotting = [(i[0],RESULTS[MODEL_INDEX][1],i[2]) for i in dbscan_clusters]
dbscan_clusters_results = [i[1] for i in dbscan_clusters]
plot_results(results = result_for_plotting, color_array=dbscan_clusters_results, force_categorical=True, point_size=5)

In [0]:
## Zoom one of the plots: 
RESULTS = TSNE_results
CLUSTER_INDEX = 3
plot_embedding(RESULTS[MODEL_INDEX]
               , fig_scale = 2.2
               , color_var_list = [dbscan_clusters[CLUSTER_INDEX][1]]
               , force_categorical=True
               , plot_centers=True
               ,  point_size = 10)

#### Profile Clusters

In [0]:
## COMPARE THE CLUSTERS 
## PICK METRICS BY WHICH CLUSTERS WOULD BE COMPARED 

DATA = descriptive_statistics
METRICS = ['cpu_run_mean', 'cpu_ready_mean']



### CREATE SUMMARY TABLE 
pd.set_option('display.max_rows', None)
pd.options.display.float_format = '{:,.3f}'.format
data = DATA[METRICS].copy()
data['dbscan_clusters'] = dbscan_clusters[CLUSTER_INDEX][1]


display(np.round(pd.melt(data, id_vars = ['dbscan_clusters']).groupby(['variable', 'dbscan_clusters']).describe(), 3))
pd.set_option('display.max_rows', 30)

In [0]:
## PARAMETERS ## 
TRANSFORMATION = return_transformer(None)
#DATA = virt_hw.loc[:, virt_hw.columns != 'os_fam'] ## it is categorical variable
DATA = correlation_data





## Plot the distribution as boxplot
clustering_df = pd.DataFrame(index = first_stage_data.index)
clustering_df['dbscan_clusters'] = dbscan_clusters[CLUSTER_INDEX][1]

data = DATA.copy()
data = TRANSFORMATION(data)
data = pd.merge(data,clustering_df, left_index=True, right_index=True)
data = pd.melt(data, id_vars = ['dbscan_clusters'])
ordr = list(sorted(set(dbscan_clusters[CLUSTER_INDEX][1])))

g = sns.FacetGrid(data, col = 'variable', col_wrap = 5, sharex = False, sharey=False, height= 5)
g.map(sns.boxplot, 'dbscan_clusters', 'value').add_legend()

plt.show()

In [0]:
## Parameters: 
DATA = correlation_data 
COLOR_VAR = 'corr_net_packetsRx_net_packetsTx'
TRANSFORMATION = return_transformer(None)
RESULTS = TSNE_results

### 
plot_embedding(  RESULTS[MODEL_INDEX]
               , fig_scale = 2
               , color_var_list = [TRANSFORMATION(DATA[COLOR_VAR].values)]
               , point_size = 20
               )

## Utility code that helps us compare the embedding generated from pca to the embedding
#pca_result_construct = ({}, first_stage_data_pca.iloc[:,0:5].values,{})
#plot_embedding(pca_result_construct, fig_scale=.8, color_var_list =  [TRANSFORMATION(DATA[COLOR_VAR].values)])

In [0]:
## CHECK ORIGINAL SIGNALS ARE DIFFERENT FOR THE DIFFERENT CLUSTERS ## 

clusters_to_compare = [1,9]
samples_per_cluster = 5
metric = 'net_packetsRx'
DATA = perf_data
sharey = False


### Fig scaling: 
num_cl_to_compare = len(clusters_to_compare)



####
sampled_df = clustering_df[clustering_df['dbscan_clusters'].isin(clusters_to_compare)].groupby('dbscan_clusters').apply(lambda x: x.sample(samples_per_cluster))
fig, axes = plt.subplots(nrows = samples_per_cluster, ncols = num_cl_to_compare, figsize = (15*num_cl_to_compare, 5*samples_per_cluster), sharey=sharey)

ix_i = 0

for i in clusters_to_compare:
    _tmp = list(sampled_df.iloc[sampled_df.index.get_level_values(0).isin([i]),:].index.get_level_values(1))
    ix_j = 0
    for j in _tmp:
        axes[ix_j, ix_i].plot(DATA[j][metric])
        axes[ix_j, ix_i].set_ylabel('vmid:'+str(j) + '('+str(i)+')')
        axes[ix_j, ix_i].yaxis.set_label_position('right')
        ix_j += 1
    ix_i += 1

## Advanced Feature Engineering

### Missing data Imputation  

The statistical tests are sensitive or do not tolerate at all missing data. 

#### Univariate imputation methods: 
*  Instance Imputation
*  Summary statistics
*  Interpolation
*  Moving average



In [0]:
## Get an intuition about the missing data
## Calculate the % of missing observations: 
dta = []
for k, data in perf_data.items():
    dta.append(data.isnull().sum().sum() / (T*D))

## Inspect the distribution of missing values 
sns.distplot(dta, kde = False, bins = 30)
plt.show()

We miss up to 5% of data as the majority of the cases lack only 1-2% 

In [0]:
## This is a toy array where we can observe  the effect of different imputation methods on the data

## These parameters control the steepness of the data
START = 1
STOP = 10
POW = 1
POLY_ORDER = 2  
TRANSFORMATION = return_transformer('None')

### 
tmp = TRANSFORMATION(pd.Series(np.random.normal(size = 100) + np.linspace(start = START, stop = STOP, num = 100)**POW))
tmp_nan = tmp[43:56].copy()
tmp[45:55] = np.nan

fig = plt.figure(figsize = (18, 6))
ax = fig.add_axes([0,0,1,1])
ax.set_title('Time series imputation methods:')
ax.plot(tmp, label = 'base ts')

base_line = plt.plot(tmp_nan, linestyle = "--", label = 'true values')
ax.plot(tmp.interpolate(method = 'linear')[44:56], label = 'linear interpolation')

## Polynomial
poly_order_label = 'poly '+ str(POLY_ORDER) +' interpolation '
plt.plot(tmp.interpolate(method = 'polynomial', order = POLY_ORDER)[44:56], label = poly_order_label)


tmp_ma_data = tmp.copy()
tmp_ma_data[44:56] = tmp.rolling(12, min_periods = 1).mean()[44:56]
ax.plot(tmp_ma_data[43:57], label = 'Simple MA')

#ax.plot(tmp.ewm(com = 0.5).mean(), label = 'EW MA') MA with Exponential decay 
#ax.plot(tmp.rolling(12, min_periods = 1).mean()[44:56], label = 'Simple MA')

plt.legend()

In [0]:
## CREATE NEW DATASET WITH IMPUTED VALUES 

IMPUTATION_METHOD = 'interpolation' # mean, median, mode, interpolation, ma
POLY_ORDER = 1 ## The order of the interpolation. 1 = linear 
MA_RANGE = 6
TRANSFORMATION = return_transformer(None)


##
OVERWRITE_MA_RANGE = True # IF the max_gap within series > MA_RANGE use MA_RANGE = max_gap.
## Calculate the largest gap 
gap_df_list = []
perf_data_no_missing = {}
i = 0
for k, data in perf_data.items():

    data_running_copy = data.copy()
    data_running_copy = TRANSFORMATION(data_running_copy)
    running_dict = {}

    for col in F:
        max_gap = data[col].isnull().astype(int).groupby(data[col].notnull().astype(int).cumsum()).sum().max()
        running_dict[col] = max_gap

        ## Apply Imputation:
        if IMPUTATION_METHOD == 'mean':
            data_running_copy[col].fillna(data_running_copy[col].mean(), inplace=True)
        elif IMPUTATION_METHOD == 'median':
            data_running_copy[col].fillna(data_running_copy[col].median(), inplace=True)
        elif IMPUTATION_METHOD == 'mode':
            data_running_copy[col].fillna(data_running_copy[col].mode(), inplace=True)
        elif IMPUTATION_METHOD == 'interpolation':
            data_running_copy[col] = data_running_copy[col].interpolate(method = 'polynomial', order = POLY_ORDER)
        elif IMPUTATION_METHOD == 'ma':
            ## Init the ma parameter
            ma_range_running = MA_RANGE

            ## Subtle hack: If the largest gap > MA_RANGE make MA_RANGE = largest gap for this metric 
            if MA_RANGE <= max_gap  and OVERWRITE_MA_RANGE:
                ma_range_running = max_gap +1 
            data_running_copy[col].fillna(data_running_copy[col].rolling(ma_range_running, min_periods = 1).mean(), inplace=True)


    gap_df_list.append(pd.DataFrame(running_dict, index = [k]))
    perf_data_no_missing[k] = data_running_copy 

    i += 1
    if i%100 == 0:
        print('Iteration', i, 'out of', N )

print('Iteration', i, 'out of', N,'\nDone')

In [0]:
## Zoom to random place in our data in order to see how our gaps have been patched: 

MISS_RANGE = 1
DATA_RANGE = np.int(144/4)

## Get random metric 
any_missings = 0
while any_missings == 0:
    metric_tmp = F[np.random.randint(low = 0, high = D)]
    vmid , data = sample_observation(data = perf_data)
    any_missings = data[metric_tmp].isnull().sum()

## Get the first and the last data 
first_ix = np.min(data[metric_tmp].index)
last_ix = np.max(data[metric_tmp].index)

##  Get the indices where the data is missing
missing_data_ix = data[metric_tmp].index[data[metric_tmp].apply(np.isnan)]
## Trick to ensure that we have gap centered between non-missing sequence
miss_date_ix = np.min([dix for dix in missing_data_ix if dix - pd.Timedelta(minutes=5 * DATA_RANGE / 2) >  first_ix and dix + pd.Timedelta(minutes=5 *  DATA_RANGE / 2) < last_ix])

## 
data_ix_reconstruct = [miss_date_ix + pd.Timedelta(minutes = 5*i) for  i in range(-DATA_RANGE, DATA_RANGE+1)]

## 
tmpdata_orig = data[data.index.isin(data_ix_reconstruct)][metric_tmp]
tmpdf = pd.DataFrame(index = tmpdata_orig.index)


##
miss_lst = []
for i in tmpdata_orig[np.isnan(tmpdata_orig)].index:
   # print(i)
    miss_lst.append(i)
    miss_lst.append(i + pd.Timedelta(minutes=5))
    miss_lst.append(i - pd.Timedelta(minutes=5))
miss_lst = list(sorted(set(miss_lst)))

## 
fig  = plt.figure(figsize = (16,4))
ax = fig.add_axes([0,0,1,1])
ax.plot(data_ix_reconstruct, TRANSFORMATION(data[data.index.isin(data_ix_reconstruct)][metric_tmp]), label = 'original')
ax.plot(miss_lst, perf_data_no_missing[vmid][perf_data_no_missing[vmid].index.isin(miss_lst)][metric_tmp], label = 'imputed', linestyle = "--")
ax.set_title('vmid: '+ str(vmid) + ' ' + metric_tmp)
plt.legend()
plt.show()
#pd.concat(gap_df_list).describe()

In [0]:
## Double check if any variable is still missing

dta_imputed = []
for k, data in perf_data_no_missing.items():
    dta_imputed.append(data.isnull().sum().sum() / (T*D))

## Check if there are instances with missing values:
still_missing = [i for i, k in enumerate(dta_imputed) if k > 1e-3]
data = [list(perf_data_no_missing.keys())[i] for i in still_missing]
print('VMs with at least 1 missing observation:' ,len(data))

#
#for i in tmp:
#    del perf_data_no_missing[i]
#print(len(perf_data_no_missing.keys()))

Virtual machines with at least 1 missing observation: 0



### Time series specific features

**_PROS_**   
More expressive  
retain the information encoded in the time domain of the data  
  
**_CONS_**   
costly to compute
hard to interpret - thus to profile cluster  
requires domain knowledge
not trivial to ensure comparability
some of the tests are very sensitive to data inconsistencies

### Spectral Density  

Another set of features that we will consider are related to the presence (or absence) of periodic patterns in the data. One way to check if series show any systematic fluctuations is through spectral decomposition of the data.  
The general idea is that any real time series (signal) can be approximated through linear combination of sine waves with different frequencies. The amplitude of these sine waves account for the variation, captured through the decomposition.  

</br>  
**Resources:**  
[Relationship between fft and psd](https://dsp.stackexchange.com/questions/24780/power-spectral-density-vs-fft-bin-magnitude)  
[Power vs. Variance](https://dsp.stackexchange.com/questions/46385/variance-in-the-time-domain-versus-variance-in-frequency-domain)  
[But what is the Fourier Transform? A visual introduction.](https://www.youtube.com/watch?v=spUNpyF58BY)  
[Introduction to the Fourier Transform](https://www.youtube.com/watch?v=1JnayXHhjlg)  
[Spectral Analysis in R](https://ms.mcmaster.ca/~bolker/eeid/2010/Ecology/Spectral.pdf)

#### Compute spectral density related features

In [0]:

## Here we define the workhorse  function that will "slide" through the 
## spectrogram and extract spectral density volumes within specific intervals 

## Arguments:
## df:          pandas data frame that contains the time series data 
## series_name:   Imputed Series 
## freq_arr: Time frequencies at which the spectrogram will be captured 
## sm_win:   Smoothing window  +/- Nsamples
## sm_fun:   The function that will be used to aggregate the values from the window  
## remove_mean: Bool indicating if mean value should be subtracted from the series
## poly_filt: filter polynomial trend 
## poly_deg: the degree of the trend (deg 1 = linear trend)
## return_series: If we want to return the frequency/ fft arrays (potentially for plotting)


def calc_PSD_features(  df
                      , series_name 
                      , freq_arr = None
                      , sm_win = 2
                      , sm_fun = np.mean
                      , remove_mean = True
                      , poly_filt = True
                      , poly_deg = 1
                      , return_series = False
                      ):
    
    data = df[series_name]
    T_data = len(data)

    ## Subtract mean:
    if remove_mean:
        data = data - np.mean(data)
    
    ## Remove linear trend: 
    if poly_filt:
        poly_coef = np.polyfit(range(T_data), data, deg = poly_deg)
    
        # This is the poly object that will actually fit the data with the poly_coef 
        poly_fit = np.poly1d(poly_coef)

        ## Apply poly filtering 
        data = data - poly_fit(range(T_data))

    ## Apply fft: 
    data_fft = np.fft.fft(data)

    ## Transform fft to psd 
    data_fft = np.power(np.abs(data_fft), 2) / T_data

    ## Include option for returning the PSD arrays for plotting:
    if return_series:
        return(data_fft)

    ## Construct the time window at which the data will be aggregated: 
    ## Keep the index in a tuple 
    
    filt_win = range(-sm_win , sm_win+1)
    per_freq = np.array(range(T_data)) 

    ## This creates tuple with freq_value (like 1, 2, 3  etc hours and their position in the array)
    time_window_array = [(ix, np.where(per_freq == ix+1) + np.array(filt_win)) for ix in freq_arr]
    
    ## Calculate Total Power of the singal / ts
    total_power = np.sum(data_fft)
    average_power = np.mean(data_fft) ## Under mean = 0 average power = variance 


    ## Calculate the captured power within the given time range 
    ## The column_name corresponds to: {variable_name}_{freq}

    data_dict = {}
    #ddict_total = '{}_uncaptured'.format(series_name)
    #data_dict[ddict_total]  = 1

    for fr, data_ix in time_window_array:
        ddict_key = '{}_pwidth_{}'.format(series_name, fr)

        data_dict[ddict_key] = sm_fun(data_fft[data_ix[data_ix > 0]]/average_power)
        #data_dict[ddict_total] -= data_dict[ddict_key] 
        #data_dict[ddict_key] = (sm_fun(data_fft[data_ix[data_ix > 0]])*2)/total_power

    
    ## Add Uncaptured variance:
    return(data_dict)

#### Inspect spectrogram for single vm

In [0]:
## Inspect the spectrogram for single VM

DATA = perf_data_no_missing
vmid = 234
metric = 'net_packetsRx'


##
data = calc_PSD_features(DATA[vmid]
                        ,series_name = metric
                        ,return_series = True
                         )
## 

fig, axes = plt.subplots(nrows = 2, figsize = (15,7))
axes[0].plot(DATA[vmid][metric] - np.mean(DATA[vmid][metric].values))
axes[0].set_ylabel('Original Series')
axes[0].yaxis.set_label_position('right')
axes[1].plot(data[:1008])
axes[1].scatter([7,14,21,28,168,336,504, 672], [0,0,0,0,0,0,0,0], c = 'red', s = 5)
axes[1].set_ylabel('Unsmoothed spectrogram')
axes[1].yaxis.set_label_position('right')
plt.show()

In [0]:
## Calculate the psd features for the entire dataset 
## Iterate over vms and variables 
## and apply function: 

########################
## TUNABLE PARAMETERS ##
########################

TRANSFORMATION = 'ln' # one of: None, 'ln', log10, 1/x, -1/x ,'cuberoot'
#FREQ_ARRAY = range(1, 7) ## We will extract the density for each hour 
FREQ_ARRAY = [7,14,21,28,168,336,672] ## We will extract the density for each hour 
SMOOTHING_WINDOW = 2 ## We will smooth them with +/- SWIN samples

REMOVE_MEAN = True
POLY_FIT = True # We will fit polynomial ...
POLY_DEG = 1 # ... of degree POLY_DEG and subtract it from the original data ultimately eliminating the respective trend 

### FREQ_ARRAY INTERPRETATION 
# FREQ_ARRAY shows where we expect to observe cycles 
# the values in FREQ_ARRAY represents number of cycles per our entire observation period 
# thus FREQ_ARRAY = 7 means that we expect to have 7 cycles/ for 1 week timeseries data that we have this corresponds to 1 cycle per day  or daily cyclicity 
# 14 = half day cyclicity / 12 hours 



###########################
## CREATE PSD DATA FRAME ##
###########################
i = 1
psd_features = []
for k, data in perf_data_no_missing.items():
    psd_res_ph = {}
    transformation = return_transformer(TRANSFORMATION)
    data = transformation(data)
    for col in data.columns: 
        psd_running = calc_PSD_features(data, col
                                      , freq_arr = FREQ_ARRAY
                                      , sm_win = SMOOTHING_WINDOW
                                      , remove_mean = REMOVE_MEAN
                                      , poly_filt = POLY_FIT
                                      , poly_deg = POLY_DEG
                                      , sm_fun = np.mean
                                        )
        dict.update(psd_res_ph, psd_running)
    
    psd_features.append(pd.DataFrame(psd_res_ph, index = [k]))

    if i%100 ==0:
        print('Iteration ', i, 'out of', N)
    
    i += 1
print('Done')
## Combine all elements into single pandas data frame :
psd_features = pd.concat(psd_features)

In [0]:
## Combine the PSD features with the ADF features  into single data frame 
psd_features.describe()

In [0]:
TRANSFORMATION = return_transformer(None)

## inspect the distribution of the features:
## NB: The data has been log transformed 

g = sns.FacetGrid(pd.melt(TRANSFORMATION(psd_features)), col = 'variable', col_wrap = 6, sharex = False, sharey=False, aspect = 1.5)
g.map(plt.hist, 'value', bins = 40)

## Second wave of modeling:

### Scale PSD features

In [0]:
#########################################
##:: psd_features DATA PREPROCESSING ::##
#########################################

## Apply transformation
TRANSFORMATION = return_transformer(None)

APPLY_STANDARD_SCALING = False

APPLY_MIN_MAX_SCALING = True
MIN_MAX_SCALER_RANGE = (-1, 1)


## We would like to retain the features
## Thus, we create hard copy of the data and operate on it:
psd_features_transformed = psd_features.copy()
psd_features_columns = psd_features.columns

psd_features_transformed[psd_features_columns] = TRANSFORMATION(psd_features_transformed[psd_features_columns])

## APPLY MIN-MAX SCALING TO DESCRIPTIVE STATISTICS DATA
if APPLY_STANDARD_SCALING:
    standart_scaler_instance = StandardScaler()
    psd_features_transformed[psd_features_columns]  = standart_scaler_instance.fit_transform(psd_features_transformed[psd_features_columns])

if APPLY_MIN_MAX_SCALING:
    psd_features_transformed[psd_features_columns] = min_max_scaler_instance.fit_transform(psd_features_transformed[psd_features_columns])


### Construct dataset for modeling

In [0]:
## CREATE FINAL DATASET ## 
## CHOOSE WHICH FEATURES TO INCLUDE FOR THE FINAL DATASET
## BY DEFAULT, WE WILL MODEL ALL FEATURES THAT WE’VE CREATED 
## HOWEVER, WE CAN MAKE MODELS OUTPUT EASIER TO INTERPRET IF WE MODEL INFORMATION ONLY FOR ONE RESOURCE TYPE (CPU or NETWORK) 


## To include columns that contains cpu_run
## descriptive_statistics_transformed.filter(regex = 'cpu_run')

## To exclude columns containing cpu_run string 
## descriptive_statistics_transformed.loc[:,~descriptive_statistics_transformed.columns.str.contains('cpu_run')]




final_data_list = [
                              descriptive_statistics_transformed
                             ,correlation_data_transformed
                             ,virt_hw_transformed
                             ,psd_features_transformed
                    ]


## To include columns that contains cpu_urn
## psd_features_transformed.filter(regex = 'cpu_run')

## To exclude columns containing cpu_run string 
## psd_features_transformed.loc[:,~psd_features_transformed.columns.str.contains('cpu_run')]

## Init empty data frame: 
final_modeling_data = pd.DataFrame(index = list(perf_data.keys()))

for i in final_data_list:
    data = i.copy()
    final_modeling_data = pd.merge(final_modeling_data, data, left_index=True, right_index=True)

##
print(final_modeling_data.shape)

### Inspect linear correlation and PC

In [0]:
### Inspect correlation:
DATA = final_modeling_data
cmap = sns.diverging_palette(220, 20, sep=20, as_cmap=True)
fig = plt.figure()
sns.clustermap(DATA.corr(), figsize = (20, 20), cmap = cmap)
plt.show()

In [0]:
### Inspect PCA: 
DATA = final_modeling_data

## One more thing to consider is the correlation among the variables: 
pca_instance = PCA().fit(DATA)
## Taking care to preserve the index for later join: 
pca_data = pd.DataFrame(pca_instance.transform(DATA), index = DATA.index)

fig, ax = plt.subplots(figsize=(20,8),nrows = 1 , ncols = 2)

sns.scatterplot(x = DATA.iloc[:,0], y = pca_data.iloc[:,1], ax = ax[0])
ax[0].set_title('Principal Components')
ax[0].set_xlabel("PC 1")
ax[0].set_ylabel("PC 2")


#tmp_plt = sns.lineplot( x= [i+1 for i in range(pca_instance.n_components_)]
#                      , y =pca_instance.explained_variance_
#                      , ax = ax[1]
#                      #, color = 'blue'
#                      )

ax[1].bar(x = [i+1 for i in range(pca_instance.n_components_)], height  = pca_instance.explained_variance_/np.sum(pca_instance.explained_variance_))

ax2 = ax[1].twinx()
ax2.plot([i+1 for i in range(pca_instance.n_components_)],  np.cumsum(pca_instance.explained_variance_)/ np.sum(pca_instance.explained_variance_), c = 'orange')
ax2.grid([])
ax2.set_ylim(bottom =0)

ax[1].set_title('Explained variance plot:')
ax[1].set_xlabel('Number of principal components')
ax[1].set_ylabel('Marginal variance')
ax2.set_ylabel('Cumulative variance')
## 

## Uniform Manifold Approximation and Projection (UMAP)  

**Resources**  
[UMAP Python Manual](https://umap-learn.readthedocs.io/en/latest/)  
[Understanding UMAP](https://pair-code.github.io/understanding-umap/)  
[How Exactly UMAP Works](https://towardsdatascience.com/how-exactly-umap-works-13e3040e1668)  

### Basic Umap Usage

In [0]:
import umap

In [0]:
## umap usage: 

DATA =  final_modeling_data

# 1. We create instance of the object
# Here we set the parameters that are internal to the algorithm

UMAP_Instance = umap.UMAP(n_components =  2         ## The dimension of the space to embed into.
                        , min_dist = 0.5            ## The effective minimum distance between embedded points. Smaller values will result in a more clustered/clumped embedding
                        , spread = 2.0              ## The effective scale of embedded points. In combination with min_dist this determines how clustered/clumped the embedded points are.
                        , n_neighbors = 15          ## The size of local neighborhood (in terms of number of neighboring sample points) used for manifold approximation.
                        , metric = 'euclidean'      ## The metric to use to compute distances in high dimensional space.  (euclidean, manhattan, cosine, )
                        ,learning_rate = 100        ## Controls the step during the gradient descent stage
                        ,n_epochs = int(1e3)        ## selected based on the size of the input dataset (200 for large datasets, 500 for small)
                        #,random_state =  500       ## Random seed 
                        ,verbose = 2                ## controls logging of the algorithm
                        ,a = None                   ## a and b are another way to control min dist and spread 
                        ,b = None                   ## 
                        )

# 2.Fit the object to the data 
result = UMAP_Instance.fit_transform(DATA)

# The result of fit_transform is ndarray with shape (NSamples, n_components)
plt.figure(figsize=(8,8))
plt.scatter(result[:, 0], result[:, 1])
plt.xlabel('X1')
plt.ylabel('X2');

### UMAP parameter tuning  

</br></br>

**Resources**  
[Basic UMAP Parameters](https://umap-learn.readthedocs.io/en/latest/parameters.html)  
[Fine-tuning UMAP Visualizations](https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html)

In [0]:
## EXPERIMENT WITH DIFFERENT UMAP SETTINGS  ## 

TUNABLE_PARAMETERS = {'n_neighbors':[10,20,30], 'min_dist':[.1,.2,.3], 'spread':[.1,.2,.3]}
DEFAULT_PARAMETERS = {}

APPLY_PCA = False
N_PC = []

MODEL = 'UMAP'
DATA = final_modeling_data


## Apply tuning:
UMAP_results = model_train(   data = DATA
                            , model = MODEL
                            , param_dict = TUNABLE_PARAMETERS
                            , apply_pca = APPLY_PCA
                            , n_pc = N_PC
                            , other_params = DEFAULT_PARAMETERS
                            )

In [0]:
## Inspect the results of the parameters

################
## PARAMETERS ##
################

RESULTS = UMAP_results
LEADING_PARAM = 'n_neighbors'
COLOR_ARRAY = None
FORCE_CATEGORICAL = False
POINT_SIZE = 5


## APPLY FUNCTION 
plot_results(results = RESULTS
            ,leading_param = LEADING_PARAM
            ,color_array = COLOR_ARRAY
            ,force_categorical = FORCE_CATEGORICAL
            ,point_size = POINT_SIZE
             )

## If single combination is being expected
##plot_embedding(RESULTS[0], fig_scale= 1.2,point_size = 5)

### Apply DBSCAN

In [0]:
## Choose one embedding to operate with 
## Zoom plot in : 
RESULTS = UMAP_results
MODEL_INDEX = 2


###
plot_embedding(RESULTS[MODEL_INDEX], fig_scale = 1.5, point_size = 4)

In [0]:
## Capture the clusters with dbscan 
TUNABLE_PARAMETERS = {'eps': [.4, .8,1.2,1.4]
                    , 'min_samples':[10, 26,28]}
DEFAULT_PARAMETERS = {}
MODEL = 'DBSCAN'

## Apply tuning:
dbscan_clusters = model_train(  data = RESULTS[MODEL_INDEX][1]
                                , model = MODEL
                                , param_dict = TUNABLE_PARAMETERS
                                , other_params = DEFAULT_PARAMETERS
                                )

In [0]:
## Plot the results from DBSCAN clustering 
result_for_plotting = [(i[0],RESULTS[MODEL_INDEX][1],i[2]) for i in dbscan_clusters]
dbscan_clusters_results = [i[1] for i in dbscan_clusters]
plot_results(results = result_for_plotting, color_array=dbscan_clusters_results, force_categorical=True, point_size=5)

In [0]:
## Zoom one of the plots: 
RESULTS = UMAP_results
CLUSTER_INDEX = 9

####
plot_embedding(RESULTS[MODEL_INDEX]
               , fig_scale = 2
               , color_var_list = [dbscan_clusters[CLUSTER_INDEX][1]]
               , force_categorical=True
               , plot_centers=True
               ,  point_size = 5)

### Profile clusters

In [0]:
## PARAMETERS ## 
TRANSFORMATION = return_transformer(None)
#DATA = virt_hw.loc[:, virt_hw.columns != 'os_fam'] ## it is categorical variable
DATA = descriptive_statistics_transformed



########
## Plot the distribution as boxplot

clustering_df = pd.DataFrame(index = final_modeling_data.index)
clustering_df['dbscan_clusters'] = dbscan_clusters[CLUSTER_INDEX][1]



data = DATA.copy()
data = TRANSFORMATION(data)
data = pd.merge(data,clustering_df, left_index=True, right_index=True)
data = pd.melt(data, id_vars = ['dbscan_clusters'])
ordr = list(sorted(set(dbscan_clusters[CLUSTER_INDEX][1])))

g = sns.FacetGrid(data, col = 'variable', col_wrap = 5, sharex = False, sharey=False, height= 5)
g.map(sns.boxplot, 'dbscan_clusters', 'value').add_legend()

plt.show()

In [0]:
## Parameters: 
DATA =  virt_hw# non_temporal_features/ virt_hw_transformed
COLOR_VAR = 'number_of_nics'
TRANSFORMATION = return_transformer(None)
RESULTS = UMAP_results



### 
plot_embedding(  RESULTS[MODEL_INDEX]
               , fig_scale = 2
               , color_var_list = [TRANSFORMATION(DATA[COLOR_VAR].values)]
               , point_size = 20
               , force_categorical = True
               )

## Utility code that helps us compare the embedding generated from pca to the embedding
#pca_result_construct = ({}, first_stage_data_pca.iloc[:,0:5].values,{})
#plot_embedding(pca_result_construct, fig_scale=.8, color_var_list =  [TRANSFORMATION(DATA[COLOR_VAR].values)])

In [0]:
## PARAMETERS ## 

clusters_to_compare = [1,4]
samples_per_cluster = 5
metric = 'net_packetsTx'
DATA = perf_data


### Fig scaling: 
num_cl_to_compare = len(clusters_to_compare)



####
sampled_df = clustering_df[clustering_df['dbscan_clusters'].isin(clusters_to_compare)].groupby('dbscan_clusters').apply(lambda x: x.sample(samples_per_cluster))

fig, axes = plt.subplots(nrows = samples_per_cluster, ncols = num_cl_to_compare, figsize = (15*num_cl_to_compare, 5*samples_per_cluster))

ix_i = 0

for i in clusters_to_compare:
    _tmp = list(sampled_df.iloc[sampled_df.index.get_level_values(0).isin([i]),:].index.get_level_values(1))
    ix_j = 0
    for j in _tmp:
        axes[ix_j, ix_i].plot(DATA[j][metric])
        axes[ix_j, ix_i].set_ylabel('vmid:'+str(j) + '('+str(i)+')')
        axes[ix_j, ix_i].yaxis.set_label_position('right')
        ix_j += 1
    ix_i += 1